# Feature Engineering

## Libraries

In [2]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data, transform_data, LOFO_GPU_Importance

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

# Feature Selection Libraries
from lofo import LOFOImportance, Dataset, plot_importance
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_absolute_error
import xgboost as xgb

## Read Data

For this pipeline, only Training Set will be used.

In [3]:
full_data = get_simplified_data()
full_data = full_data[full_data['Set']=='Train']
y_train = pd.read_csv('Data/Y_train.csv')

As done in the other Notebooks, we will transform the Column 'Time' to Datetime format and set as the index of the dataset.

In [4]:
full_data['Time'] = pd.to_datetime(full_data['Time'],dayfirst=True)
full_data = full_data.set_index('Time')

In [5]:
full_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [6]:
WF = 'WF1'
data = full_data[full_data['WF']==WF]
y_train = y_train[y_train['ID'].isin(data['ID'])]

In [8]:
full_data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train
...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,37371,WF6,-0.868480,-5.454230,0.660126,-0.827276,274.113000,0.000000,Train
2019-01-15 21:00:00,37372,WF6,-0.453421,-3.824979,0.420679,-0.751297,274.520365,0.000000,Train
2019-01-15 22:00:00,37373,WF6,-0.874800,-4.534290,0.144120,-0.699230,273.261000,0.000000,Train


## Feature Creation

### Wind Speed Vector

In [6]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])
feature_data = feature_data.drop(['U_100m','V_100m','U_10m','V_10m'],axis=1)

Changing Reference for Negative Angles:

In [7]:
feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

Using Wind Speed and Direction instead of U and V, we will create some variables over the Numerical Variables from the simplified data.

In [8]:
feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m'] = 360 - feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m']

In [9]:
features = ['T', 'CLCT', 'Wind Speed 100m','Wind Direction 100m', 'Wind Speed 10m', 'Wind Direction 10m']

### Time-Relative Variables

Here, we get Values for Last Week and Month for each Numerical Feature.

In [10]:
for column in features:
    feature_data[column + '_last_week'] = feature_data[column].shift(7)
    feature_data[column + '_last_month'] = feature_data[column].shift(30)

Get the Number of Month:

In [11]:
feature_data['Month_Number'] = feature_data.index.month

Month Statistics:

In [12]:
mean = feature_data.groupby('Month_Number').mean()[features]
median = feature_data.groupby('Month_Number').median()[features]
variance = feature_data.groupby('Month_Number').var()[features]

In [13]:
mean.columns = mean.columns + '_Month_Mean'
median.columns = median.columns + '_Month_Median'
variance.columns = variance.columns + '_Month_Variance'

In [14]:
feature_data = feature_data.merge(mean,on='Month_Number',how='left')
feature_data = feature_data.merge(median,on='Month_Number',how='left')
feature_data = feature_data.merge(variance,on='Month_Number',how='left')

In [15]:
feature_data.index = data.index

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [16]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

### Rolling Window Variables

### Wavelet Transformations (Check)

## Tsfresh

Now we use Tsfresh, a Python Library that automates Feature Engineering for Time Series Data. We generate new features for all the columns on the Simplified Data, as done below.

In [17]:
data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]

In [18]:
tsfresh_data = pd.DataFrame()
for variable in ['U_100m','V_100m','U_10m','V_10m','T','CLCT']: 
    df_shift, y = make_forecasting_frame(data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
    X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,show_warnings=False,n_jobs=3)
    X['Feature'] = variable
    tsfresh_data = tsfresh_data.append(X)

Feature Extraction: 100%|██████████| 15/15 [01:48<00:00,  7.22s/it]


Process tsfresh_data to pass column 'Features' to the other columns

In [19]:
tsfresh_data = tsfresh_data.pivot(columns='Feature')

In [20]:
tsfresh_data.columns = tsfresh_data.columns.map('{0[0]}|{0[1]}'.format)

In [21]:
tsfresh_data.head()

,value__abs_energy|CLCT,value__abs_energy|T,value__abs_energy|U_100m,value__abs_energy|U_10m,value__abs_energy|V_100m,value__abs_energy|V_10m,value__absolute_sum_of_changes|CLCT,value__absolute_sum_of_changes|T,value__absolute_sum_of_changes|U_100m,value__absolute_sum_of_changes|U_10m,...,value__variance|U_100m,value__variance|U_10m,value__variance|V_100m,value__variance|V_10m,value__variance_larger_than_standard_deviation|CLCT,value__variance_larger_than_standard_deviation|T,value__variance_larger_than_standard_deviation|U_100m,value__variance_larger_than_standard_deviation|U_10m,value__variance_larger_than_standard_deviation|V_100m,value__variance_larger_than_standard_deviation|V_10m
id,,,,,,,,,,,,,,,,,,,,,
2018-05-01 02:00:00,6813.370572,82047.873600,5.055752,1.574030,10.613261,0.083919,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 03:00:00,16811.539518,163992.661200,10.982542,7.778652,12.704466,0.254793,17.447701,0.180,0.186000,1.236305,...,0.008649,0.382112,0.820564,0.003824,1.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 04:00:00,26487.652359,246117.891825,12.472335,8.772845,12.775686,2.257409,19.071310,0.495,1.399929,2.730120,...,0.284984,0.425138,1.513166,0.253942,1.0,0.0,0.0,0.0,1.0,0.0
2018-05-01 05:00:00,35486.186521,327217.540225,26.210477,9.248391,51.431749,3.181777,22.577941,2.290,6.327000,3.037614,...,6.250924,0.467797,5.034489,0.202684,1.0,0.0,1.0,0.0,1.0,0.0
2018-05-01 06:00:00,44684.124127,408135.031825,40.752497,9.333069,81.075418,3.268781,23.623216,2.610,6.433900,3.436218,...,8.045999,0.556415,5.149121,0.198239,1.0,0.0,1.0,0.0,1.0,0.0


In [22]:
tsfresh_data = tsfresh_data.fillna(0)

## Feature Selection

In [23]:
final_features = feature_data.merge(tsfresh_data,left_on=feature_data.index,right_on=tsfresh_data.index,how='left')

final_features = final_features.rename({'key_0':'Date'},axis=1)

In [24]:
features = final_features.drop(['ID','WF','Set','Date'],axis=1).columns

In [25]:
final_features = transform_data(final_features[features])
y_train = transform_data(y_train[['Production']])

In [26]:
model  = xgb.XGBRegressor(tree_method='gpu_hist')

In [27]:
importance_df = LOFO_GPU_Importance(final_features,y_train['Production'],features,model)

1/4585 13.622494 s/it
2/4585 13.484766 s/it
3/4585 13.591627 s/it
4/4585 13.662627 s/it
5/4585 13.704747 s/it
6/4585 14.294350 s/it
7/4585 14.720705 s/it
8/4585 14.55463 s/it
9/4585 14.18513 s/it
10/4585 13.887112 s/it
11/4585 13.785861 s/it
12/4585 13.618014 s/it
13/4585 13.781347 s/it
14/4585 13.676971 s/it
15/4585 13.677553 s/it
16/4585 13.772459 s/it
17/4585 13.802408 s/it
18/4585 13.727986 s/it
19/4585 13.808075 s/it
20/4585 14.770457 s/it
21/4585 13.844336 s/it
22/4585 13.955288 s/it
Exception ignored in: <function Booster.__del__ at 0x000002912BB24CA8>
Traceback (most recent call last):
  File "C:\Users\andre_\anaconda3\lib\site-packages\xgboost\core.py", line 1065, in __del__
    _check_call(_LIB.XGBoosterFree(self.handle))
KeyboardInterrupt
23/4585 14.416404 s/it
24/4585 14.965475 s/it


In [1]:
final_features

NameError: name 'final_features' is not defined